# Best Winners and Worst Losers

In [7]:
{"tags": ["hide-cell"]}
import warnings; warnings.simplefilter('ignore')

# Modules
import numpy as np
import pandas as pd
from espn_api.football import League
from espn_api.football import constant

import altair as alt
from altair import datum


# League Parameters
league_id = 298982
# year = 2019
swid = '{D825016D-4C3D-4575-B33C-2C2277B026F0}'
espn_s2 = 'AEAmlIpZf7Zd7LTyvSFyl9k3zui2t4hQwlVtJM8WK3Lmx33eWNAUq32gR9NK98ZjqXWIEsK3NETxdtcctstCwGu45Mx9QOM9wIeB9KBTALOs8wg512Me2GSTnw3MCQL8bAeTp16w0xkggMxdmDGX9BX6nS2dKDx5OfjEIFLsgnntd3CW%2BmYsMnCywwMpQQZQkigiNNM54cM7OmivIq2kGkZY%2BJEhquYe%2FHnSRhBa9f052nnEmYn0fAzax8RHr3boSL9UASeMp2B8dLubfQd83Bj%2B%2BwBJPpeGyxi36lwJPtMXjg%3D%3D'


# Helper Functions, Accumulate Data
league_start = 2011

# Helper Functions
def find_original_lineup(lineup):
    og_lineup_pos = [player.lineupSlot for player in lineup]
    og_lineup_name = [player.name for player in lineup]

    og_lineup_df = pd.DataFrame({
        'og_lineup_pos': og_lineup_pos,
        'og_lineup_name': og_lineup_name
    })

    # remove non-starters
    og_lineup_df = og_lineup_df[~og_lineup_df['og_lineup_pos'].isin(['BE', 'IR'])].reset_index(drop = True)

    # and thier score
    og_score = np.sum([player.points for player in lineup if player.lineupSlot not in ['BE', 'IR']])
    projected_score = np.sum([player.projected_points for player in lineup if player.lineupSlot not in ['BE', 'IR']])

    return og_lineup_df, og_score, projected_score

def find_optimal_lineup(week, positions, lineup):
    # empty lists to append to
    optimal_positions = []
    optimal_players = []

    # 1. outer loop: go through all the positions
    for position in positions:

        # skip the non-starters
        if position in ['BE', 'IR']:
            continue
        if positions[position] != 0:
            n_players = positions[position]
            
            # save the positions and counts
            optimal_positions.append(np.repeat(position, n_players))

            # 2. inner loop: find highest player for each position
            eligible_players = []
            for player in lineup:
                if position in player.eligibleSlots:

                    # save the player name
                    eligible_players.append(player)

            # sort players by points scored; keep roster available number
            eligible_players = sorted(eligible_players, key = lambda x: x.points, reverse = True)[:n_players]
            optimal_players.extend(eligible_players)

            # remove players already used
            for player in eligible_players:
                lineup.remove(player)

    optimal_lineup_df = pd.DataFrame({
        'optimal_positions': sum([pos.tolist() for pos in optimal_positions], []),
        'optimal_players': [player.name for player in optimal_players]
    })
    optimal_lineup_df['week'] = week

    # output the score as well
    optimal_score = np.sum(player.points for player in optimal_players)

    return optimal_lineup_df, optimal_score

def combine_og_optimal(og_lineup_df, optimal_lineup_df):
    og_lineup_df['og_lineup_pos'] = og_lineup_df['og_lineup_pos'].astype('category')
    og_lineup_df['og_lineup_pos'] = og_lineup_df['og_lineup_pos'].cat.set_categories(optimal_lineup_df['optimal_positions'].unique())
    og_lineup_df = og_lineup_df.sort_values('og_lineup_pos').reset_index(drop = True)

    comp_df = pd.concat([optimal_lineup_df, og_lineup_df], axis = 1)

    return comp_df

def gather_all_optimal(league, all_comps, all_scores):
    num_weeks = league.current_week
    # no need to loop for positions
    positions = league.settings.position_slot_counts

    # loop through all the weeks we have so far
    for j in range(num_weeks - 1):

        # Access a week of box scores
        week_box_scores = league.box_scores(week = j + 1)
        
        # loop through all the home & away teams
        num_of_matchups = len(week_box_scores)

        for i in range(num_of_matchups):
            home_team = week_box_scores[i].home_team
            home_lineup = week_box_scores[i].home_lineup

            manager = home_team.owners[0]['firstName'] + ' ' + home_team.owners[0]['lastName']
            
            # make a copy to not reset values
            home_lineup2 = home_lineup.copy()

            # find optimal & original lineups & scores for the team
            optimal_lineup_df, optimal_score = find_optimal_lineup(week = j + 1,
                                                                   positions = positions,
                                                                   lineup = home_lineup2)
            og_lineup_df, og_score, projected_score = find_original_lineup(home_lineup)

            # another data frame to host scores
            oo_score_df = pd.DataFrame({
                'manager': np.repeat(manager, 3),
                'week': np.repeat(j + 1, 3),
                'score': [og_score, optimal_score - og_score, projected_score],
                'type': ['original', 'optimal', 'projected']
            })

            optimal_lineup_df['manager'] = manager

            # join the dataframes
            comp_df = combine_og_optimal(og_lineup_df, optimal_lineup_df)

            # append the dataframes
            all_comps.append(comp_df)
            all_scores.append(oo_score_df)

        for i in range(num_of_matchups):
            away_team = week_box_scores[i].away_team
            away_lineup = week_box_scores[i].away_lineup

            manager = away_team.owners[0]['firstName'] + ' ' + away_team.owners[0]['lastName']
            
            # make a copy to not reset values
            away_lineup2 = away_lineup.copy()

            # find optimal & original lineups & scores for the team
            optimal_lineup_df, optimal_score = find_optimal_lineup(week = j + 1,
                                                                   positions = positions,
                                                                   lineup = away_lineup2)
            og_lineup_df, og_score, projected_score = find_original_lineup(away_lineup)

            # another data frame to host scores
            oo_score_df = pd.DataFrame({
                'manager': np.repeat(manager, 3),
                'week': np.repeat(j + 1, 3),
                'score': [og_score, optimal_score - og_score, projected_score],
                'type': ['original', 'optimal', 'projected']
            })

            optimal_lineup_df['manager'] = manager

            # join the dataframes
            comp_df = combine_og_optimal(og_lineup_df, optimal_lineup_df)

            # append the dataframes
            all_comps.append(comp_df)
            all_scores.append(oo_score_df)

    all_comps_df = pd.concat(all_comps).reset_index(drop = True)
    all_scores_df = pd.concat(all_scores).reset_index(drop = True)

    return all_comps_df, all_scores_df

def create_master_data():
    # get all the possible years' worth of data
    # current_year = datetime.date.today().year
    current_year = 2024
    all_years = np.arange(league_start, current_year + 1)

    # remove 2018 as its a problem (TO FIX)
    index_2018 = np.where(all_years == 2018)
    if index_2018[0].shape[0] == 1:
        all_years = np.delete(all_years, index_2018)

    # get data for all the years in raw format
    leagues = []
    for year in all_years:
        league = League(league_id, year, espn_s2, swid)
        leagues.append(league)

    # loop through each league and grab game-level data
    scores_df = []

    # use the loop to grab the final standings too
    standings_df = []

    # use the loop to grab the acquisition counts too
    acq_df = []

    # use the loop to grab the draft results
    draft_board = []

    # use the loop to grab player data
    player_lookup = []

    # use the loop to grab optimal comparison data
    all_comps = []
    all_scores = []

    # outer loop to loop through all the leagues/years
    for league in leagues:
        season_weeks = league.settings.reg_season_count
        teams = league.teams
        team_ids = [team.team_id for team in teams]

        # metrics for the draft board
        num_picks = len(league.draft)

        # inner loop to loop through the teams in each year
        for id in team_ids:
            team_data = league.get_team_data(id)

            # some managers' data has been purged
            if team_data is None:
                continue
            else:
                opponent_list = pd.DataFrame({'opponent': [opp.owners[0]['firstName'] + ' ' + opp.owners[0]['lastName'] for opp in team_data.schedule],
                                              'outcome': team_data.outcomes,
                                              'points_for': team_data.scores,
                                              'mov': team_data.mov})
                opponent_list['manager'] = team_data.owners[0]['firstName'] + ' ' + team_data.owners[0]['lastName']
                opponent_list['game_type'] = 'postseason'
                opponent_list['week'] = np.arange(opponent_list.shape[0]) + 1
                scores_df.append(opponent_list)
                opponent_list.iloc[:season_weeks, 5] = 'season'
                opponent_list['year'] = league.year

                # acquisition data
                team_id = team_data.owners[0]['firstName'] + ' ' + team_data.owners[0]['lastName']
                pickups = team_data.acquisitions
                trades = team_data.trades
                faab_used = team_data.acquisition_budget_spent
                acq_list = pd.DataFrame({'team_id': [team_id],
                                         'pickups': [pickups],
                                         'trades': [trades],
                                         'faab_used': [faab_used],
                                         'year': [league.year]})
                acq_df.append(acq_list)

        # accumulate the standings from each year
        standings = [team.owners[0]['firstName'] + ' ' + team.owners[0]['lastName'] for team in league.standings()]
        standings_num = np.arange(len(standings)) + 1
        df_standings = pd.DataFrame({'Manager': standings, 'Result': standings_num})
        df_standings['Year'] = league.year
        standings_df.append(df_standings)

        # collect draft values here
        for i in range(num_picks):
            draft_pick = league.draft[i]
            round = draft_pick.round_num
            pick = draft_pick.round_pick
            player = draft_pick.playerName
            manager = manager = draft_pick.team.owners[0]['firstName'] + ' ' + draft_pick.team.owners[0]['lastName']

            # combine all into an appendable data frame
            pick_df = pd.DataFrame({'round': [round],
                                    'pick': [pick],
                                    'player': [player],
                                    'manager': [manager],
                                    'year': [league.year]})

            draft_board.append(pick_df)

        # collect player data here
        dat = league.espn_request.get_pro_players()
        for player_info in dat:
            if 'eligibleSlots' not in player_info.keys():
                continue
            # position_id = player_info['eligibleSlots'][0]

            # filter for only first eligible player spot that isnt a combo
            for pos in player_info['eligibleSlots']:
                if (pos != 25 and '/' not in constant.POSITION_MAP[pos]) or '/' in player_info['fullName']:
                    position = constant.POSITION_MAP[pos]
                    break

            # remainder of player data
            player = player_info['fullName']
            pi = pd.DataFrame({'position': [position],
                               'player': [player],
                               'year': [league.year]})
            player_lookup.append(pi)

        if league.year == current_year:
            all_comps_df, all_scores_df = gather_all_optimal(league, all_comps, all_scores)

    # all the concatenations
    scores_df = pd.concat(scores_df)
    scores_df['manager'] = scores_df['manager'].str.title()
    scores_df['opponent'] = scores_df['opponent'].str.title()
    scores_df['points_against'] = scores_df['points_for'] - scores_df['mov']

    standings_df = pd.concat(standings_df)
    standings_df['Manager'] = standings_df['Manager'].str.title()

    acq_df = pd.concat(acq_df)
    acq_df['team_id'] = acq_df['team_id'].str.title()

    draft_board_df = pd.concat(draft_board)
    draft_board_df['manager'] = draft_board_df['manager'].str.title()

    all_scores_df['manager'] = all_scores_df['manager'].str.title()
    all_comps_df['manager'] = all_comps_df['manager'].str.title()

    # concatenate and remove duplicates (defense, etc.)
    player_lookup_df = pd.concat(player_lookup)
    player_lookup_df['row_num'] = player_lookup_df.groupby(['player', 'year'], as_index = False).cumcount() + 1
    player_lookup_df = player_lookup_df[player_lookup_df['row_num'] == 1]

    # add draft position to the draft
    draft_board_df['player_pos'] = draft_board_df['player'] + ' (' + draft_board_df['round'].astype(str) + '.' + draft_board_df['pick'].astype(str) + ')'

    # add playing position to the draft
    draft_board_df = pd.merge(draft_board_df,
                              player_lookup_df,
                              how = 'left',
                              on = ['player', 'year'])

    return scores_df, standings_df, acq_df, draft_board_df, all_comps_df, all_scores_df

scores_df, standings_df, acq_df, draft_board_df, all_comps_df, all_scores_df = create_master_data()

In [2]:
{"tags": ["hide-cell"]}

# Data Manipulation
scores_df['divisor'] = np.where(
    (scores_df['game_type'] == 'postseason') &
    (scores_df['year'] != 2024), 2, 1
)

def anti_join(df1, df2, left_on, right_on):
    """Perform an anti-join between two DataFrames."""
    merged = pd.merge(df1, df2, left_on = left_on, right_on = right_on, how = 'left', indicator = True)
    return merged[merged['_merge'] == 'left_only'].drop(columns = '_merge')

# split the 2 week playoffs into 2 separate observations
scores_df1 = scores_df[scores_df['divisor'] == 1]
scores_df2 = scores_df[scores_df['divisor'] == 2]
scores_df2 = pd.concat([scores_df2, scores_df2])
scores_df2['points_for'] = scores_df2['points_for'] / 2

# stack em back together
scores = pd.concat([scores_df1, scores_df2])

# means for all teams
overall_means = scores.loc[:, ['year', 'points_for']].groupby('year', as_index = False).mean()
overall_means['manager'] = 'Overall'

# means for non-championship teams
means_no_champ = (
    anti_join(
        scores,
        standings_df.groupby('Year').head(1),
        left_on = ['manager', 'year'],
        right_on = ['Manager', 'Year']
    )
    .loc[:, ['year', 'points_for']]
    .groupby('year', as_index = False)
    .mean()
    .reset_index(drop = True)
)
means_no_champ['manager'] = 'Overall - No Champion'

# means for non-sacko teams
means_no_sacko = (
    anti_join(
        scores, 
        standings_df.groupby('Year').tail(1), 
        left_on = ['manager', 'year'], 
        right_on = ['Manager', 'Year']
    )
    .loc[:, ['year', 'points_for']]
    .groupby('year', as_index = False)
    .mean()
    .reset_index(drop = True)
)
means_no_sacko['manager'] = 'Overall - No Sacko'

# means for non-championship teams & non-sacko teams
means_no_sacko_champ = (
    anti_join(
        scores,
        pd.concat([standings_df.groupby('Year').head(1), 
                   standings_df.groupby('Year').tail(1)]),
        left_on = ['manager', 'year'], 
        right_on = ['Manager', 'Year']
    )
    .loc[:, ['year', 'points_for']]
    .groupby('year', as_index = False)
    .mean()
    .reset_index(drop = True)
)
means_no_sacko_champ['manager'] = 'Overall - No Sacko/Champ'

# means for champions
means_champ = (
    pd.merge(
        scores,
        standings_df.groupby('Year').head(1),
        left_on = ['manager', 'year'],
        right_on = ['Manager', 'Year']
    )
    .loc[:, ['year', 'points_for']]
    .groupby('year', as_index = False)
    .mean()
    .reset_index(drop = True)
)
means_champ['manager'] = 'Champions'

# means for sackos
means_sacko = (
    pd.merge(
        scores,
        standings_df.groupby('Year').tail(1),
        left_on = ['manager', 'year'],
        right_on = ['Manager', 'Year']
    )
    .loc[:, ['year', 'points_for']]
    .groupby('year', as_index = False)
    .mean()
    .reset_index(drop = True)
)
means_sacko['manager'] = 'Sackos'

# mean for every manager every year
yearly_means = scores.loc[:, ['manager', 'year', 'points_for']].groupby(['manager', 'year'], as_index = False).mean()
yearly_means2 = yearly_means.copy()
yearly_means['manager'] = 'Individual'

# ll_means_df = pd.concat([overall_means, means_no_champ, means_no_sacko, means_no_sacko_champ, yearly_means, means_sacko, means_champ])
all_means_df = pd.concat([overall_means, means_no_sacko_champ, yearly_means, means_sacko, means_champ])
all_means_df2 = pd.concat([overall_means, means_no_sacko_champ, yearly_means2, means_sacko, means_champ])

## Scatterplot
Multiple metrics plotted here. For now, I don't have access to 2018; it should change soon. The scale is from 60 to 120 to spread out numbers a bit better. I've plotted the average score of:
1. The champion (green)
2. The sacko (red)
3. The league overall (blue)
4. The league without the sacko nor the champ (orange)
5. Every single manager (light grey)

In [3]:
{"tags": ["hide-input"]}

scoring_chart = (
    alt.Chart(
        all_means_df,
        title = 'Scatterplot of Mean Points'
    ).mark_circle(size = 60).encode(
        x = alt.X('year:O').title('Year'),
        y = alt.Y(
            'points_for:Q',
            scale = alt.Scale(domain = [60, 120])
        ).title('Average Points'),
        color = alt.Color(
            'manager:N',
            scale = alt.Scale(
                domain = ['Champions', 'Individual', 'Overall', 'Overall - No Sacko/Champ', 'Sackos'],
                range = ['green', '#D3D3D3', 'blue', 'orange', 'red']
            )
        ).title('Score Type')
        # color = 'manager:N'
    )
)
scoring_chart

alt.Chart(...)

## Boxplot
Plotted: box and whisker plot. Champions (green) and Sackos (red) with a bar to designate the distribution of the remaining managers.

In [4]:
{"tags": ["hide-input"]}

base = (
    alt.Chart(
        all_means_df,
        title = 'Distribution of Average Scores'
    ).encode(
        x = alt.X('year:O').title('Year'),
        y = alt.Y(
            'points_for:Q',
            scale = alt.Scale(domain = [60, 120])
        ).title('Average Points'),
        color = alt.Color(
            'manager:N',
            scale = alt.Scale(
                domain = ['Champions', 'Individual', 'Sackos'],
                range = ['green', '#D3D3D3', 'red']
            )
        ).title('Score Type')
    )
)

base.mark_boxplot(extent = 'min-max').transform_filter((datum.manager == 'Individual')) + base.mark_circle(size = 60).transform_filter((datum.manager == 'Sackos') | (datum.manager == 'Champions'))


alt.LayerChart(...)

## In Table Form
The numbers behind these plots.

## Best Champs

In [5]:
{"tags": ["hide-input"]}

greatest_champs = (
    means_no_sacko_champ
    .rename(columns = {'points_for': 'mean_points'})
    .merge(standings_df.groupby('Year').head(1),
           how = 'left',
           left_on = 'year',
           right_on = 'Year')
    .drop(['manager', 'Result', 'Year'], axis = 1)
    .merge(means_champ.drop('manager', axis = 1),
           how = 'left',
           on = 'year')
)
greatest_champs['difference'] = greatest_champs['points_for'] - greatest_champs['mean_points']
greatest_champs = (
    greatest_champs
    .sort_values('difference', ascending = False)
)

greatest_champs.loc[:, ['year', 'Manager', 'mean_points', 'points_for', 'difference']]
greatest_champs.columns = ['Year', 'Manager', 'League Average', 'Manager Average', 'Difference']
greatest_champs

,Year,Manager,League Average,Manager Average,Difference
9,2021,90.242647,Sirpi Nackeeran,110.352941,20.110294
3,2014,86.011765,Brian Babcock,101.941176,15.929412
11,2023,90.161765,Brian Babcock,102.705882,12.544118
8,2020,94.970588,Alec Sa,107.058824,12.088235
2,2013,92.691176,Ryan Last Name,103.823529,11.132353
5,2016,90.426471,Michael Mount,100.235294,9.808824
6,2017,86.875000,Brian Babcock,94.176471,7.301471
12,2024,90.064706,Nikhil Tellakula,97.235294,7.170588
0,2011,95.598039,Alec Sa,101.882353,6.284314
7,2019,91.551471,Jordan Fladger,96.882353,5.330882


## Worst Sackos

In [6]:
{"tags": ["hide-input"]}

biggest_losers = (
    means_no_sacko_champ
    .rename(columns = {'points_for': 'mean_points'})
    .merge(standings_df.groupby('Year').tail(1),
           how = 'left',
           left_on = 'year',
           right_on = 'Year')
    .drop(['manager', 'Result', 'Year'], axis = 1)
    .merge(means_sacko.drop('manager', axis = 1),
           how = 'left',
           on = 'year')
)
biggest_losers['difference'] = biggest_losers['points_for'] - biggest_losers['mean_points']
biggest_losers = (
    biggest_losers
    .sort_values('difference', ascending = True)
)

biggest_losers.loc[:, ['year', 'Manager', 'mean_points', 'points_for', 'difference']]
biggest_losers.columns = ['Year', 'Manager', 'League Average', 'Manager Average', 'Difference']
biggest_losers

,Year,Manager,League Average,Manager Average,Difference
2,2013,92.691176,Drew Mount,76.411765,-16.279412
6,2017,86.875000,Sirpi Nackeeran,75.647059,-11.227941
7,2019,91.551471,Michael Oatman,80.588235,-10.963235
5,2016,90.426471,Sirpi Nackeeran,79.882353,-10.544118
11,2023,90.161765,Brian Badillo,80.823529,-9.338235
0,2011,95.598039,Steven Tindle,87.882353,-7.715686
8,2020,94.970588,Casey Cabrales,88.117647,-6.852941
10,2022,90.323529,Brian Badillo,85.588235,-4.735294
12,2024,90.064706,Casey Cabrales,85.764706,-4.300000
9,2021,90.242647,Alec Sa,88.529412,-1.713235
